In [1]:
# I decide to work in greedy way(start from less feature and try to add another feature)
# and I decide to eveluate only weight MEA on kaggle website
# for model that use to evaluate I only Decision tree

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
from datetime import datetime
import datetime
from sklearn.model_selection import KFold
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
train = pd.read_csv('./train.csv')
feature = pd.read_csv('./features.csv')
test = pd.read_csv('./test.csv')
train['Date'] = pd.to_datetime(train['Date'])
train['Week'] = train['Date'].dt.week
test['Date'] = pd.to_datetime(test['Date'])
test['Week'] = test['Date'].dt.week
feature['Date'] = pd.to_datetime(feature['Date'])
train['IsHoliday'] = (train[['IsHoliday']]== True).astype(int)
test['IsHoliday'] = (test[['IsHoliday']]== True).astype(int)
train_with_feature = pd.merge_asof(train, feature, on='Store',by='Date')
test_with_feature = pd.merge_asof(test, feature, on='Store',by='Date')

In [4]:
train_with_feature

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Week,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,0,5,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,1,6,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,0,7,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,0,8,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,0,9,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
5,1,1,2010-03-12,21043.39,0,10,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,False
6,1,1,2010-03-19,22136.64,0,11,54.58,2.720,NaN,NaN,NaN,NaN,NaN,211.215635,8.106,False
7,1,1,2010-03-26,26229.21,0,12,51.45,2.732,NaN,NaN,NaN,NaN,NaN,211.018042,8.106,False
8,1,1,2010-04-02,57258.43,0,13,62.27,2.719,NaN,NaN,NaN,NaN,NaN,210.820450,7.808,False
9,1,1,2010-04-09,42960.91,0,14,65.86,2.770,NaN,NaN,NaN,NaN,NaN,210.622857,7.808,False


In [5]:
# as for first select feature I look back to knowing data
# I select value that kindly relate with sales as base so I select date
# and I decide to create other feature from date
# so we have week and holiday as base and create other feature from it

In [6]:
start_feature_train = train_with_feature[['Store','Dept','Date','Weekly_Sales','IsHoliday_x','Week']]
start_feature_test = test_with_feature[['Store','Dept','Date','IsHoliday_x','Week']]

In [7]:
def create_prenext_week(data):
    dataset = data.copy()
    dataset['preweek'] = dataset['Week']-1
    dataset.loc[dataset['preweek'] == 0, 'preweek'] = 52
    dataset['nextweek'] = dataset['Week']+1
    dataset.loc[dataset['nextweek'] == 53, 'nextweek'] = 1
    PreNext1week = dataset.copy()
    dataset['pre2week'] = dataset['preweek']-1
    dataset.loc[dataset['pre2week'] == 0, 'pre2week'] = 52
    dataset['next2week'] = dataset['nextweek']+1
    dataset.loc[dataset['next2week'] == 53, 'next2week'] = 1
    PreNext2week = dataset.copy()
    return(PreNext1week,PreNext2week)

In [8]:
# from plot Weekly_Sales and week some sale peak slide to beside week 1-2 week in years so I decide to create column nextweek, next2week,preweek and pre2week
train1week,train2week = create_prenext_week(start_feature_train)
test1week,test2week = create_prenext_week(start_feature_test)

In [9]:
def Tree_Realtest(name,train,test):
    name = './'+name+'_Tree.csv'
    f = open(name,'w')
    f.write('Id,Weekly_Sales\n')
    for store in test.Store.unique():
        select_store_test = test.loc[test['Store'] == store]
        select_store_train = train.loc[train['Store'] == store]
        for dept in select_store_test.Dept.unique():
            select_dept_test = select_store_test.loc[select_store_test['Dept'] == dept]
            dates = select_dept_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
            select_dept_test = select_dept_test.drop(columns=['Store', 'Dept','Date']).reset_index()
            select_dept_train = select_store_train.loc[select_store_train['Dept'] == dept]
            select_dept_train = select_dept_train.drop(columns=['Store', 'Dept','Date']).reset_index()
            if len(select_dept_train.index) > 0: # handle no train data for test dept ex. store 5 dept 99
                X_train = select_dept_train.drop(columns=['Weekly_Sales'])
                Y_train = select_dept_train['Weekly_Sales']
                X_test = select_dept_test
                clf = DecisionTreeRegressor()
                clf.fit(X_train,Y_train)
                test_y = clf.predict(X_test)
                for i in range(len(test_y)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = test_y[i]
                    f.write('%s,%s\n'%(Id,sales))
            else:
                for i in range(len(select_dept_test)):
                    Id = str(store)+'_'+str(dept)+'_'+str(dates[i])
                    sales = 0
                    f.write('%s,%s\n'%(Id,sales))
    f.close()

In [10]:
Tree_Realtest('week_1',train1week,test1week) #5178.89486
Tree_Realtest('week_2',train2week,test2week) #5136.95584
Tree_Realtest('base',start_feature_train,start_feature_test) #5350.64879
# Try to create feature for month level but seem not work

In [11]:
# from plot Weekly_Sales and Holydays have many outlier and seem not good to use as feature so I try remove it.
train2week.drop(columns=['IsHoliday_x'])
test2week.drop(columns=['IsHoliday_x'])
Tree_Realtest('week_2',train2week,test2week) #5088.47749

In [12]:
# from result remove holiday dataset have better result so I try to create holiday myself base on Federal Holidays data: http://www.ca3.uscourts.gov/2018-federal-holidays
# and add other festival that not include in holidays ex. super_bowl

In [13]:
def get_holiday(date):
    super_bowl = ['2010-02-12','2011-02-11','2012-02-10','2013-02-08']
    labor = ['2010-09-10','2011-09-09','2012-09-07']
    thank = ['2010-11-26','2011-11-25','2012-11-23']
    chris = ['2010-12-31','2011-12-30','2012-12-28']
    columbus =['2010-10-15','2011-10-14','2012-10-12']
    veterans = ['2010-11-12','2011-11-11','2012-11-16']
    independence = ['2010-7-9','2011-7-8','2012-7-6','2013-7-5']
    memorial = ['2010-6-4','2011-6-3','2012-6-1','2013-5-31']
    washington = ['2010-2-19','2011-2-25','2012-2-24','2013-2-22']
    if date in super_bowl:
        return [0,0,0,0,0,0,0,0,1]
    elif date in labor:
        return [0,0,0,0,0,0,0,1,0]
    elif date in thank:
        return [0,0,0,0,0,0,1,0,0]
    elif date in chris: 
        return [0,0,0,0,0,1,0,0,0]
    elif date in columbus: 
        return [0,0,0,0,1,0,0,0,0]
    elif date in veterans:
        return [0,0,0,1,0,0,0,0,0]
    elif date in independence:
        return [0,0,1,0,0,0,0,0,0]
    elif date in memorial:
        return [0,1,0,0,0,0,0,0,0]
    elif date in washington:
        return [1,0,0,0,0,0,0,0,0]
    else:
        return [0,0,0,0,0,0,0,0,0]

In [14]:
def multi_holiday(data):
    dataset = data.copy()
    holiday = []
    dates = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday.append(get_holiday(str(dates[i])))
    holiday = pd.DataFrame(holiday)
    holiday.columns = ['super_bowl', 'labor','thank','chris','columbus','veterans','independence','memorial','washington']
    dataset = pd.merge(dataset, holiday, left_index=True, right_index=True)
    normal_holiday = dataset.copy()
    holiday_pre1 = []
    dates = data['Date'].apply(lambda x: (x-datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_pre1.append(get_holiday(str(dates[i])))
    holiday_next1 = []
    dates = data['Date'].apply(lambda x: (x+datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_next1.append(get_holiday(str(dates[i])))
    holiday_pre1 = pd.DataFrame(holiday_pre1)
    holiday_next1 = pd.DataFrame(holiday_next1)
    holiday_pre1.columns = ['super_bowl_p1', 'labor_p1','thank_p1','chris_p1','columbus_p1','veterans_p1','independence_p1','memorial_p1','washington_p1']
    holiday_next1.columns = ['super_bowl_n1', 'labor_n1','thank_n1','chris_n1','columbus_n1','veterans_n1','independence_n1','memorial_n1','washington_n1']
    dataset = pd.merge(dataset, holiday_pre1, left_index=True, right_index=True)
    dataset = pd.merge(dataset, holiday_next1, left_index=True, right_index=True)
    PreNext1week = dataset.copy()
    holiday_pre2 = []
    dates = data['Date'].apply(lambda x: (x-datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_pre2.append(get_holiday(str(dates[i])))
    holiday_next2 = []
    dates = data['Date'].apply(lambda x: (x+datetime.timedelta(days=7)).strftime('%Y-%m-%d')).values
    for i in range(len(dates)):
        holiday_next2.append(get_holiday(str(dates[i])))
    holiday_pre2 = pd.DataFrame(holiday_pre2)
    holiday_next2 = pd.DataFrame(holiday_next2)
    holiday_pre2.columns = ['super_bowl_p2', 'labor_p2','thank_p2','chris_p2','columbus_p2','veterans_p2','independence_p2','memorial_p2','washington_p2']
    holiday_next2.columns = ['super_bowl_n2', 'labor_n2','thank_n2','chris_n2','columbus_n2','veterans_n2','independence_n2','memorial_n2','washington_n2']
    dataset = pd.merge(dataset, holiday_pre2, left_index=True, right_index=True)
    dataset = pd.merge(dataset, holiday_next2, left_index=True, right_index=True)
    PreNext2week = dataset.copy()
    return(normal_holiday,PreNext1week,PreNext2week)

In [15]:
start_feature_train = train_with_feature[['Store','Dept','Date','Weekly_Sales','Week']]
start_feature_test = test_with_feature[['Store','Dept','Date','Week']]
train1week,train2week = create_prenext_week(start_feature_train)
test1week,test2week = create_prenext_week(start_feature_test)
normaltrain,train1week,train2week = multi_holiday(train2week)
normaltest,test1week,test2week = multi_holiday(test2week)

In [16]:
# from old score 5088.47749
Tree_Realtest('holiday_1',train1week,test1week) #5084.24131
Tree_Realtest('holiday_2',train2week,test2week) #5020.45259
Tree_Realtest('normal',normaltrain,normaltest) #5118.58290